In [1]:
from estnltk import Text
from estnltk.converters import json_to_text
from estnltk.taggers import Retagger
from estnltk.taggers import CompoundTokenTagger

import os
import re

In [2]:
class TokenSplitter( Retagger ):
    """Splits tokens into smaller tokens based on regular expression patterns.""" 
    conf_param = ['patterns', 'break_group_name']
    
    def __init__(self, patterns, break_group_name:str='end'):
        # Set input/output layers
        self.input_layers = ['tokens']
        self.output_layer = 'tokens'
        self.output_attributes = ()
        # Set other configuration parameters
        if not (isinstance(break_group_name, str) and len(break_group_name) > 0):
            raise TypeError('(!) break_group_name should be a non-empty string.')
        self.break_group_name = break_group_name
        # Assert that all patterns are regular expressions in the valid format
        if not isinstance(patterns, list):
            raise TypeError('(!) patterns should be a list of compiled regular expressions.')
        # TODO: we use an adhoc way to verify that patterns are regular expressions 
        #       because there seems to be no common way of doing it both in py35 
        #       and py36
        for pat in patterns:
            # Check for the existence of methods/attributes
            has_match   = callable(getattr(pat, "match", None))
            has_search  = callable(getattr(pat, "search", None))
            has_pattern = getattr(pat, "pattern", None) is not None
            for (k,v) in (('method match()',has_match),\
                          ('method search()',has_search),\
                          ('attribute pattern',has_pattern)):
                if v is False:
                    raise TypeError('(!) Unexpected regex pattern: {!r} is missing {}.'.format(pat, k))
            symbolic_groups = pat.groupindex
            if self.break_group_name not in symbolic_groups.keys():
                raise TypeError('(!) Pattern {!r} is missing symbolic group named {!r}.'.format(pat, self.break_group_name))
        self.patterns = patterns

    def _change_layer(self, text, layers, status):
        # Get changeble layer
        changeble_layer = layers[self.output_layer]
        # Iterate over tokens
        add_spans    = []
        remove_spans = []
        for span in changeble_layer:
            token_str = text.text[span.start:span.end]
            for pat in self.patterns:
                m = pat.search(token_str)
                if m:
                    break_group_end = m.end( self.break_group_name )
                    if break_group_end > -1 and \
                       break_group_end > 0  and \
                       break_group_end < len(token_str):
                        # Make the split
                        add_spans.append( (span.start, span.start+break_group_end) )
                        add_spans.append( (span.start+break_group_end, span.end) )
                        remove_spans.append( span )
                        # Once a token has been split, then break and move on to 
                        # the next token ...
                        break
        if add_spans:
            assert len(remove_spans) > 0
            for old_span in remove_spans:
                changeble_layer.remove_span( old_span )
            for new_span in add_spans:
                changeble_layer.add_annotation( new_span )

In [3]:
token_splitter = TokenSplitter(patterns=[re.compile(r'(?P<end>[A-ZÕÄÖÜ]{1}\w+)[A-ZÕÄÖÜ]{1}\w+'),\
                                         re.compile(r'(?P<end>Piebenomme)metsawaht'),\
                                         re.compile(r'(?P<end>maa)peal'),\
                                         re.compile(r'(?P<end>reppi)käest'),\
                                         re.compile(r'(?P<end>Kiidjerwelt)J'),\
                                         re.compile(r'(?P<end>Ameljanow)Persitski'),\
                                         re.compile(r'(?P<end>mõistmas)Mihkel'),\
                                         re.compile(r'(?P<end>tema)Käkk'),\
                                         re.compile(r'(?P<end>Ahjawalla)liikmed'),\
                                         re.compile(r'(?P<end>kohtumees)A'),\
                                         re.compile(r'(?P<end>Pechmann)x'),\
                                         re.compile(r'(?P<end>pölli)Anni'),\
                                         re.compile(r'(?P<end>külla)Rauba'),\
                                         re.compile(r'(?P<end>kohtowannem)Jaak'),\
                                         re.compile(r'(?P<end>rannast)Leno'),\
                                         re.compile(r'(?P<end>wallast)Kiiwita'),\
                                         re.compile(r'(?P<end>wallas)Kristjan'),\
                                         re.compile(r'(?P<end>Pedoson)rahul'),\
                                         re.compile(r'(?P<end>pere)Jaan'),\
                                         re.compile(r'(?P<end>kohtu)poolest'),\
                                         re.compile(r'(?P<end>Kurrista)kaudo'),\
                                         re.compile(r'(?P<end>mölder)Gottlieb'),\
                                         re.compile(r'(?P<end>wöörmündri)Jaan'),\
                                         re.compile(r'(?P<end>Oinas)ja'),\
                                         re.compile(r'(?P<end>ette)Leenu'),\
                                         re.compile(r'(?P<end>Tommingas)peab'),\
                                         re.compile(r'(?P<end>wäljaja)Kotlep'),\
                                         re.compile(r'(?P<end>pea)A'),\
                                         re.compile(r'(?P<end>talumees)Nikolai')])

In [4]:
files = {}

with open('divided_corpus.txt', 'r', encoding = 'UTF-8') as f:
    txt = f.readlines()

for fileName in txt:
    file, subdistribution = fileName.split(":")
    files[file] = subdistribution.rstrip("\n")

In [5]:
files_not_working = ['J2rva_Tyri_V22tsa_id22177_1911a.json', \
                     'J2rva_Tyri_V22tsa_id18538_1894a.json', \
                     'J2rva_Tyri_V22tsa_id22155_1911a.json', \
                     'Saare_Kihelkonna_Kotlandi_id18845_1865a.json', \
                     'P2rnu_Halliste_Abja_id257_1844a.json', \
                     'Saare_Kaarma_Loona_id7575_1899a.json', \
                     'J2rva_Tyri_V22tsa_id22266_1913a.json']

In [6]:
%%time
training_texts = []
filenames = {key: value for key, value in files.items() if value in ('1', '2', '3', '4')}
for filename in filenames:
    with open('./vallakohtufailid_json/' + str(filename), 'r', encoding='UTF-8') as file:
        if filename in files_not_working:
            continue
        else:
            training_texts.append(json_to_text(file.read()).tag_layer(['sentences', 'morph_analysis']))

CPU times: user 2min 3s, sys: 2.85 s, total: 2min 5s
Wall time: 2min 7s


In [7]:
from estnltk.taggers.estner.ner_trainer import NerTrainer
from estnltk.taggers.estner.model_storage_util import ModelStorageUtil
from estnltk.core import DEFAULT_PY3_NER_MODEL_DIR

model_dir=DEFAULT_PY3_NER_MODEL_DIR
modelUtil = ModelStorageUtil(model_dir)
nersettings = modelUtil.load_settings()
trainer = NerTrainer(nersettings)

In [8]:
%%time
trainer.train( training_texts, layer='gold_wordner', model_dir='test' )

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 346163
Seconds required: 2.987

Stochastic Gradient Descent (SGD)
c2: 0.001000
max_iterations: 1000
period: 10
delta: 0.000001

Calibrating the learning rate (eta)
calibration.eta: 0.100000
calibration.rate: 2.000000
calibration.samples: 1000
calibration.candidates: 10
calibration.max_trials: 20
Initial loss: 32455.512517
Trial #1 (eta = 0.100000): 2799.600043
Trial #2 (eta = 0.200000): 4119.565209
Trial #3 (eta = 0.400000): 8561.548498
Trial #4 (eta = 0.800000): 17339.056734
Trial #5 (eta = 1.600000): 36579.034982 (worse)
Trial #6 (eta = 0.050000): 2474.419152
Trial #7 (eta = 0.025000): 2645.977417
Trial #8 (eta = 0.012500): 3060.319073
Trial #9 (eta = 0.006250): 3661.918528
Trial #10 (eta = 0.003125): 4489.437405
Trial #11 (eta = 0.001563): 5702.986127
Trial #12 (eta = 0.000781): 7580.143885
Trial 

***** Epoch #38 *****
Loss: 458.478717
Improvement ratio: 0.296488
Feature L2-norm: 106.345452
Learning rate (eta): 0.049811
Total number of feature updates: 593332
Seconds required for this iteration: 0.607

***** Epoch #39 *****
Loss: 436.858799
Improvement ratio: 0.347190
Feature L2-norm: 106.976032
Learning rate (eta): 0.049806
Total number of feature updates: 608946
Seconds required for this iteration: 0.602

***** Epoch #40 *****
Loss: 421.401630
Improvement ratio: 0.347264
Feature L2-norm: 107.579146
Learning rate (eta): 0.049801
Total number of feature updates: 624560
Seconds required for this iteration: 0.618

***** Epoch #41 *****
Loss: 428.935369
Improvement ratio: 0.272400
Feature L2-norm: 108.179763
Learning rate (eta): 0.049796
Total number of feature updates: 640174
Seconds required for this iteration: 0.606

***** Epoch #42 *****
Loss: 405.246311
Improvement ratio: 0.317421
Feature L2-norm: 108.758502
Learning rate (eta): 0.049791
Total number of feature updates: 655788